### Topic modeling
Extracting topics from the reddit messages 

##### Imports 

In [136]:
# Imports
from bertopic import BERTopic
import pandas as pd
import numpy as np
import nltk 

##### Data 

Loading the data 

In [137]:
# getting subset of data 
messages_df = pd.read_csv("reddit_22_51/messages.csv", sep="\t")
messages_df.head()

,id,user,text
0,j0s252k,HexagonOfVirtue,"im gonna find it just to check, it's not the ..."
1,j0s25h2,Teephex,According to you criticizing and being skeptic...
2,j0s25ht,1platesquat,Gotcha. Can you explain to me why your opinion...
3,j0s25l5,YouLostTheGame,"Euros, which some argue is actually harder tha..."
4,j0s25nr,HMID_Delenda_Est,You've been sounding more like PunishedSubSist...


Info on the data 

In [138]:
# info on data
column_list = messages_df.columns
shape = messages_df.shape

print("columns: ", column_list)
print("shape ", shape)

columns:  Index(['id', 'user', 'text'], dtype='object')
shape  (290898, 3)


**Data cleaning: stop word removal**

Stop word are very frequent words e.g. “the” and “a” that can impact the topics generated by the bertopic model due to their high frequency across most documents/texts - can be removed to get clearer more informative topics 

In [139]:
# METHOD 1: use a predefined list of stop words 
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

def remove_stop_words(text, stop_words): 
    words = text.split() 
    # Use a list comprehension to remove stop words 
    filtered_words = [word for word in words if word.lower() not in stop_words] 
  
  # Join the filtered words back into a sentence 
    return ' '.join(filtered_words)

messages_df_sw = messages_df.copy()
messages_df_sw["text"] = messages_df_sw["text"].apply(lambda row: remove_stop_words(row, stop_words))
messages_df_sw.head()

[nltk_data] Downloading package stopwords to /home/jnye/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,user,text
0,j0s252k,HexagonOfVirtue,"im gonna find check, ai art one right? trigger..."
1,j0s25h2,Teephex,According criticizing skeptical active politic...
2,j0s25ht,1platesquat,Gotcha. explain opinion important me?
3,j0s25l5,YouLostTheGame,"Euros, argue actually harder WC"
4,j0s25nr,HMID_Delenda_Est,sounding like PunishedSubSister tbh


In [140]:
# METHOD 2: remove most frequent words found in the data 
most_frequent_words = pd.Series(' '.join(messages_df['text']).lower().split()).value_counts()[:100].index.tolist()

messages_df_sw2 = messages_df.copy()
messages_df_sw2["text"] = messages_df_sw2["text"].apply(lambda row: remove_stop_words(row, most_frequent_words))

Convert data to list of strings (input needed by bertopic)

In [141]:
# convert to list of strings (input needed by bertopic model)
messages_list = messages_df["text"].astype(str).tolist()
len(messages_list)

290898

Get subset of data to work with 

In [142]:
# subset of data - 5000 rows
messages_subset = messages_list[:5000]
messages_subset

['im gonna find check, ai art one right? triggered rare',
 'According criticizing skeptical active politican mental disorder?',
 'Gotcha. explain opinion important me?',
 'Euros, argue actually harder WC',
 'sounding like PunishedSubSister tbh',
 'hate people race hate people born',
 'communism abysmal failure ever single instances attempted? Yeah. Hans would agree.',
 'made genuine effort try estimate cost moving affordable place?',
 'implemented rule saying link social media sites (Instagram, facebook, Mastodon, etc)',
 'Yes. Every single human earth eats tiny amounts poison regular basis. know know that? literally everything poisonous depending dose. Water poisonous drink enough it.',
 'set base couple days. Go read Volume 2 AFH 10-222 see done.',
 'welcome opinion.',
 'Literally least worst Trump adjacent person could sell to.',
 'Speech thought. Thought police.',
 'know happened school system.',
 'Thread OP: "so need landlords?" Me: "unironically yes" tbh long one rental company g

##### Bertopic model 

In [143]:
topic_model = BERTopic()
# topic_model = BERTopic.load("MaartenGr/BERTopic_Wikipedia")

In [144]:
# fitting the bertopic model 
topic_model_fitted = topic_model.fit(messages_subset)

In [145]:
# parameters of the general model
topic_model_fitted.get_params()

{'calculate_probabilities': False,
 'ctfidf_model': ClassTfidfTransformer(),
 'embedding_model': <bertopic.backend._sentencetransformers.SentenceTransformerBackend at 0x7fe66e65abc0>,
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'language': 'english',
 'low_memory': False,
 'min_topic_size': 10,
 'n_gram_range': (1, 1),
 'nr_topics': None,
 'representation_model': None,
 'seed_topic_list': None,
 'top_n_words': 10,
 'umap_model': UMAP(angular_rp_forest=True, low_memory=False, metric='cosine', min_dist=0.0, n_components=5, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True}),
 'vectorizer_model': CountVectorizer(),
 'verbose': False,
 'zeroshot_min_similarity': 0.7,
 'zeroshot_topic_list': None}

In [146]:
# get info on topics (names, important words, representative reddit message/document)
topic_info = topic_model_fitted.get_topic_info()
topic_info.to_csv("topic_info.csv", sep="\t", index=False)

In [ ]:
# get the top words for each topic id (form the topic names)
topic_model_fitted.get_topics()

{-1: [('like', np.float64(0.00831196820904529)),
  ('people', np.float64(0.0080854059802529)),
  ('one', np.float64(0.007265190753662957)),
  ('think', np.float64(0.007032515345787205)),
  ('would', np.float64(0.006894981509575474)),
  ('im', np.float64(0.00674484288688382)),
  ('get', np.float64(0.006462665311152999)),
  ('it', np.float64(0.006222985431660049)),
  ('its', np.float64(0.005785928689648847)),
  ('much', np.float64(0.005603209780318356))],
 0: [('thank', np.float64(0.03066904994864566)),
  ('mad', np.float64(0.029073189116356597)),
  ('lol', np.float64(0.028465633959276756)),
  ('sorry', np.float64(0.02520713448799441)),
  ('im', np.float64(0.023950912445824434)),
  ('thats', np.float64(0.02366487676584492)),
  ('sub', np.float64(0.022024115563244632)),
  ('comment', np.float64(0.02190646424903261)),
  ('you', np.float64(0.02147343606580249)),
  ('oh', np.float64(0.02146892335445048))],
 1: [('argument', np.float64(0.03045023455050473)),
  ('faith', np.float64(0.026581834